<a href="https://colab.research.google.com/github/poiut/Generators/blob/master/Copy_of_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generators and Iterators

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/notebooks/generator.ipynb)

This chapter introduces generator functions, which are functions that yield a stream of values, rather than returning a single value.

To demonstrate their use, we'll explore Cartesian products, permutations, and combinations, using playing cards as an example.

## Generators

As a first example, we'll write a generator function that generates the playing cards in a standard 52-card deck.
This example is inspired by an example in Peter Norvig's ["A Concrete Introduction to Probability (using Python)"](https://nbviewer.ipython.org/url/norvig.com/ipython/Probability.ipynb).

Here are Unicode strings that represent the set of suits and the set of ranks.

In [ ]:
suits = u'♠♥♦♣'
ranks = u'AKQJ⑽98765432'

And here's a nested for loop that enumerates all pairings of a rank with a suit.

In [ ]:
for rank in ranks:
    for suit in suits:
        print(rank+suit, end=' ') 

A♠ A♥ A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

This set of pairs is the [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of the set of ranks and the set of suits.

The following function encapsulates the loops and uses the `yield` statement to generate a stream of cards.

In [ ]:
def card_generator(ranks, suits):
    for rank in ranks:
        for suit in suits:
            yield rank+suit

Because this function includes a `yield` statement, it is a generator function. When we call it, the return value is a generator object.

In [ ]:
it = card_generator(ranks, suits)
it

<generator object card_generator at 0x7fe820790120>

The generator object is iterable, so we can use `next` to get the first element of the stream.

In [ ]:
next(it)

'A♠'

The first time we call `next`, the function runs until it hits the `yield` statement.
If we call `next` again, the function resumes from where it left off and runs until it hits the `yield` statement again. 

In [ ]:
next(it)

'A♥'

Because `it` is iterable, we can use it in a for loop to enumerate the remaining pairs.

In [ ]:
for card in it:
    print(card, end=' ')

A♦ A♣ K♠ K♥ K♦ K♣ Q♠ Q♥ Q♦ Q♣ J♠ J♥ J♦ J♣ ⑽♠ ⑽♥ ⑽♦ ⑽♣ 9♠ 9♥ 9♦ 9♣ 8♠ 8♥ 8♦ 8♣ 7♠ 7♥ 7♦ 7♣ 6♠ 6♥ 6♦ 6♣ 5♠ 5♥ 5♦ 5♣ 4♠ 4♥ 4♦ 4♣ 3♠ 3♥ 3♦ 3♣ 2♠ 2♥ 2♦ 2♣ 

When the flow of control reaches the end of the function, the generator object raises and exception, which causes the for loop to end.

## itertools

The `itertools` library provides function for working with iterators, including `product`, which is a generator function that takes iterators as arguments at yields their Cartesian product.
We'll use `itertools.product` in the next few sections; then we'll see how to implement it.

Here's a loop that uses `itertools.product` to generate the playing cards again.

In [ ]:
from itertools import product

for t in product(ranks, suits):
    card = ''.join(t)
    print(card, end=' ')        

NameError: ignored

**Exercise:** Encapsulate the previous loop in a generator function called `card_generator2` that yields the playing  cards. Then call your function and use it to print the cards.

## Enumerating all pairs

Now that we have playing cards, let's deal a few hands. In fact, let's deal all the hands.

First, I'll create two card generators.

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

Now we can use `product` to generate all pairs of cards.

In [ ]:
for hand in product(it1, it2):
    print(hand)

To check whether it's working correctly, it will be useful to count the number of elements in an iterator, which is what `ilen` does.
This idiom is discussed [on Stack Overflow](https://stackoverflow.com/questions/390852/is-there-any-built-in-way-to-get-the-length-of-an-iterable-in-python).

In [ ]:
def ilen(it):
    return sum(1 for _ in it)

Now we can use it to count the pairs of cards.

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)
ilen(product(it1, it2))

2704

If things have gone according to plan, the number of pairs should be $52^2$.

In [ ]:
52**2

2704

Notice that we have to create new card iterators every time, because once they are used up, they behave like an empty list.
Here's what happens if we try to use them again.

In [ ]:
ilen(product(it1, it2))

0

That's also why we had to create two card iterators.
If you create one and try to use it twice, it doesn't work.

In [ ]:
it = card_generator(ranks, suits)
ilen(product(it, it))

0

However, you can get around this limitation by calling `product` with the `repeat` argument, which makes it possible to use a single iterator to generate a Cartesian product.

In [ ]:
it = card_generator(ranks, suits)
ilen(product(it, repeat=2))

2704

## Permutations

In the previous section, you might have noticed that some of the hands we generated are impossible because they contain the same card more than once.

One way to solve this problem is to generate all pairs and then eliminate the ones that contain duplicates.

In [ ]:
it = card_generator(ranks, suits)

for hand in product(it, repeat=2):
    if len(hand) == len(set(hand)):
        print(hand)

**Exercise:** Write a generator function called `permutations` that takes an iterator and and integer, `r`, as arguments. It should generate tuples that represent all subsets of the elements in the iterator with size `r` and no duplicates.

Test your function by generating and printing all hands with two distinct cards.
Then use `ilen` to count how many there are, and confirm that it's `52 * 51`.

In [ ]:
from itertools import product

def card_gen(rnks, sts):
  for pair in product(sts, str(rnks)):
    card = ''.join(pair)
    yield card

In [ ]:
itr1 = card_gen(ranks, suits)

In [ ]:


def permutations( itr, r):
  
  res1 = list(product(itr, repeat=r))
  
  to_remove = {}
  for tup in res1:
    for x in tup:
      x_index = tup.index(x)
      if x in tup[x_index+1: ] and tup not in to_remove:
        to_remove[tup] = res1.index(tup) 
    
  for x in res1[::-1]:
    if x in to_remove:
      res1.pop(to_remove[x])
  
  return res1


res2 = permutations( it1, 2)
"""
counter = 0
for x in res2:
  if counter % 50 == 0:
    print(x)
  counter += 1
  """



'\ncounter = 0\nfor x in res2:\n  if counter % 50 == 0:\n    print(x)\n  counter += 1\n  '

In [ ]:
len(res2)

2652

In [ ]:
combo1_lst = combinations(res2)
print(len( combo1_lst))

1326


In [ ]:
ilen = sum(1 for x in res2)
print(ilen)

2652


The `itertools` library provides a function called `permutations` that does the same thing.

In [ ]:
import itertools

it = card_generator(ranks, suits)
ilen(itertools.permutations(it, 2))

2652

## Combinations

At this point we are generating legitimate hands in the sense that the same card never appears twice.
But we end up generating the same hand more than once, in the sense that the order of the cards does not matter.
So we consider `(card1, card2)` to be the same hand as `(card2, card1)`.
To avoid that, we can generate all permutations and then filter out the ones that are not in sorted order.

It doesn't really matter which order is considered "sorted"; it's just a way to choose one ordering we consider "canonical".

That's what the following loop does.

In [ ]:
it = card_generator(ranks, suits)

for hand in permutations(it, r=2):
    if list(hand) == sorted(hand):
        print(hand)

**Exercise:** Write a generator function called `combinations` that takes an iterator and and integer, `r`, as arguments. It should generate tuples that represent all *sorted* subsets of the elements in the iterator with size `r` and no duplicates.

Test your function by generating and printing all hands with two distinct cards.
Then use `ilen` to count how many there are, and confirm that it's `52 * 51 / 2`.

In [ ]:

from itertools import product, combinations

def ccombinations(itr, r):
  itr_prod_lst = list(product(itr, repeat=r))
  #print(f" original list = {itr_prod_lst}")
  indices_to_pop = []

  for i in range(len(itr_prod_lst)):

    if all( itr_prod_lst[i][0] == y for y in itr_prod_lst[i][1: ]):
      indices_to_pop.append(i)
                
    else:
      for y in itr_prod_lst[i+1: ]:
        if sorted(itr_prod_lst[i]) == sorted(y):
          indices_to_pop.append(i)
    # Two sorted there because the ambition 
    # was to make it for more

  sorted_indices = sorted(indices_to_pop)        
  for indx in sorted_indices[::-1]:
    itr_prod_lst.pop(indx)
  return itr_prod_lst



In [ ]:
test1 = ['a', 'b', 'c']

for n in range(2, 7):
  lst = [ x for x in range(n) ]
  print( len( list(combinations(lst, 2)) ) == len( list(ccombinations(lst, 2)) ))

True
True
True
True
True


In [ ]:
def ilen2( v):
  res = sum(1 for x in v)
  return res

The `itertools` library provides a function called `combinations` that does the same thing.

In [ ]:
import itertools

it = card_generator(ranks, suits)
ilen(itertools.combinations(it, 2))

NameError: ignored

## Generating hands

We can use `combinations ` to write a generator that yields all valid hands with `n` playing cards, where "valid" means that the cards are in sorted order with no duplicates.

In [ ]:
from itertools import product

def card_gen(rnks, sts):
  for pair in product( ranks, suits):
    card = ''.join(pair)
    yield card

In [ ]:
itr = card_gen(ranks, suits)
ilen2(itr)

52

In [ ]:
from itertools import combinations

def hand_generator(n=2):
    it = card_gen(ranks, suits)
    for hand in combinations(it, n):
        yield hand

In [ ]:
res = list(hand_generator())
res[10]

('A♠', 'Q♣')

In [ ]:
ilen2(hand_generator())

1326

If you ever find yourself looping through an iterator and yielding all of the elements, you can simplify the code using `yield from`.

In [ ]:
def hand_generator(n=2):
    it = card_gen(ranks, suits)
    yield from combinations(it, n)

In [ ]:
ilen2(hand_generator(2))

1326

In [ ]:
def ilen(it):
    return sum(1 for _ in it)

In [ ]:
ilen(hand_generator(5))

2598960

Now let's see how many hands there are with 3, 4, and (maybe) 5 cards.

In [ ]:
ilen2(hand_generator(3))

22100

In [ ]:
ilen2(hand_generator(4))

270725

I'm not patient enough to let this one finish.

In [ ]:
#ilen2(hand_generator(5))

But if we only care about the number of combinations, we can use [`math.comb`](https://docs.python.org/3/library/math.html).

In [ ]:
from math import comb

comb(52, 5)

2598960

## How many flushes?

In poker, a "flush" is a hand where all cards have the same suit.
To check whether a hand is a flush, it is convenient to extract the suit part of the cards and make a set.

In [ ]:
it = hand_generator(4)
hand = next(it)
hand

('A♠', 'A♥', 'A♦', 'A♣')

In [ ]:
type(hand)

tuple

In [ ]:
it_lst = list(it)



In [ ]:
#for x in it_lst[0]:  
#  print(x[1])
it_lst[0]

('A♠', 'A♥', 'A♦', 'K♠')

In [ ]:
suit = it_lst[0][1][1]   # first index = hand, second = card, third = suit
#if all(card[1] == suit for card in hand):
#  return True
suit

'♥'

In [ ]:
suits_st = set(card[1] for card in hand)

In [ ]:
for x in range(7):
  print(it_lst[x])

**Exercise:** Write a function called `is_flush` that takes a hand as an argument and returns `True` if all cards are the same suit.

Then write a generator function called `flush_generator` that takes an integer `n` and return all hands with `n` cards that are flushes.

What fraction of hands with 3, 4, and 5 cards are flushes?

In [ ]:
def is_flush(handy):
  suit = handy[1][1]
  if all(card[1] == suit for card in handy):
    return True
  
  

In [ ]:
is_flush(hand2)

True

In [ ]:
hand2 = ('A'+suit, 'K'+suit, 'Q'+suit, str(5)+suit)
hand2

('A♥', 'K♥', 'Q♥', '5♥')

In [ ]:
from itertools import product

def hand_gen2(n):
  cards = card_gen(ranks, suits)  # the deck
  for hand in product(cards, repeat=n):
    yield hand


In [ ]:
hands = hand_gen2(2)   # len = 2704

In [ ]:
count = 0
for hand in hand_gen2(2):
  if is_flush(hand):
    count += 1

In [ ]:
count/2704

0.25

In [ ]:
def flush_generator(n):
  
  for hand in hand_gen2(n):
    if is_flush(hand):
      yield hand


In [ ]:
res_2 = list(flush_generator(2))    # = 676 flushes, fraction = .25

In [ ]:
res3 = flush_generator(3)   # = 8788 fkushes, total = 140608, frction 0.0625

In [ ]:
res_4 = sum(1 for x in flush_generator(4))  # = 114244, total = 7311616, fraction = 0.015625

In [ ]:
res_5 = sum( 1 for x in flush_generator(5))   # stopped at the minute

In [ ]:
#tot4 = sum(1 for x in hand_gen2(4))
114244/tot4

0.015625

## Write your own product

So far we've been using `itertools.product`, but in the same way we wrote `permutations` and `combinations`, we can write our own `product`.

If there are only two iterators, we can do it with nested `for` loops.

In [ ]:
def product2(it1, it2):
    for x in it1:
        for y in it2:
            yield x, y

So we can generate the cards like this.

In [ ]:
for t in product2(ranks, suits):
    card = ''.join(t)
    print(card, end=' ')

Now, we might be tempted to write two-card hands like this.

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

for hand in product2(it1, it2):
    print(hand)

# has been consumed and has to be called again

# New Section

But that doesn't work; it only generates the first 52 pairs.
Before you go on, see if you can figure out why.

We can solve this problem by making each iterator into a tuple; then we can loop through them more than once.
The price we pay is that we have to store all of the elements of the iterators.

In [ ]:
def product2(it1, it2):
    t1 = tuple(it1)
    t2 = tuple(it2)
    for x in t1:
        for y in t2:
            yield x, y

This version of `product2` works if the arguments are iterators.

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

for hand in product2(it1, it2):
    print(hand)

In [ ]:
it1 = card_generator(ranks, suits)
it2 = card_generator(ranks, suits)

ilen2(product2(it1, it2))

Now let's take it up a notch. What if you want the product of more than two iterators.
The version of `product` we got from `itertools` can handle this case.

In [ ]:
import itertools

for pair in itertools.product(range(2), range(3), range(4)):
    print(pair)

**Exercise:** Write a generator function that takes an arbitrary number of iterables and yields their Cartesian product. Compare the results to `itertools.product`.

Hint: I found it easiest to write this recursively.

In [1]:
def arg_maker2():
  return [1, 2], ['a', 'b'], ['A', 'B'], [11, 22]

def arg_maker3():
  return [1, 2, 3], ['a', 'b', 'c'], ['A', 'B', 'C'], [11, 22, 33]



In [17]:


def cartesian_arb_iterative( *args):
  # The cartesian product is a row

  columns = len(args)

  rows = 1
  for arg in args:
    rows *= len(arg)

  lst1 = [ [ None for y in range(columns) ] for x in range(rows) ]

  multipliers, prdct = [1], 1
  
  work_lst_args = args[::-1]
  for itr in work_lst_args[:-1]:
    prdct *= len(itr)
    multipliers.append(prdct)

  multipliers.reverse()
  
  args_mult = list(zip(args, multipliers))

  for iterable, current_multr in args_mult[::-1]:
    
    row = 0
    column = args.index(iterable)
    print(f"\n")   
    
    while row < rows:            # the outside loop

      for element in iterable:
        for m in range(current_multr):
          #print(f" row = {row}, col= {column}, element = {element}") 
          lst1[row ][column] = element
          row += 1
       
  for row in lst1:
    #print(row)
    yield row



  # yippy!

In [9]:
l1, l2, l3, l4 = arg_maker2()
ll1, ll2, ll3, ll4 = arg_maker3()

In [10]:
l1, ll2, l4, l2

([1, 2], ['a', 'b', 'c'], [11, 22], ['a', 'b'])

In [16]:
res = cartesian_arb_iterative( l1, l2)





[1, 'a']
[1, 'b']
[2, 'a']
[2, 'b']


In [15]:
res = cartesian_arb_iterative(l1, ll2, l4)







[1, 'a', 11]
[1, 'a', 22]
[1, 'b', 11]
[1, 'b', 22]
[1, 'c', 11]
[1, 'c', 22]
[2, 'a', 11]
[2, 'a', 22]
[2, 'b', 11]
[2, 'b', 22]
[2, 'c', 11]
[2, 'c', 22]


In [7]:
list2 = [ [str(row)+str(col) for col in range(5)] for row in range(3)]

In [3]:
for r in list2:
  print(r)

['00', '01', '02', '03', '04']
['10', '11', '12', '13', '14']
['20', '21', '22', '23', '24']


*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)